## Observations and Insights 

In [29]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as st

# Study data files
mouse_metadata_path = "data/Mouse_metadata.csv"
study_results_path = "data/Study_results.csv"

# Read the mouse data and the study results
mouse_metadata = pd.read_csv(mouse_metadata_path)
study_results = pd.read_csv(study_results_path)

# Combine the data into a single dataset
mouse_data_merged = mouse_metadata.merge(study_results, left_on="Mouse ID", right_on="Mouse ID", how="inner")

# Display the data table for preview
mouse_data_merged.head()


,Mouse ID,Drug Regimen,Sex,Age_months,Weight (g),Timepoint,Tumor Volume (mm3),Metastatic Sites
0,k403,Ramicane,Male,21,16,0,45.000000,0
1,k403,Ramicane,Male,21,16,5,38.825898,0
2,k403,Ramicane,Male,21,16,10,35.014271,1
3,k403,Ramicane,Male,21,16,15,34.223992,1
4,k403,Ramicane,Male,21,16,20,32.997729,1


In [34]:
# Checking the number of mice.
meta_count = mouse_metadata["Mouse ID"].nunique()
study_count = study_results["Mouse ID"].nunique()
merged_count = mouse_data_merged["Mouse ID"].nunique()
print(f'Mouse counts: meta: {meta_count}, study: {study_count}, merged {merged_count}')

Mouse counts: meta: 249, study: 249, merged 249


In [ ]:
# From instructions: "Before beginning the analysis, check the data for any mouse ID with duplicate time points and remove 
# any data associated with that mouse ID"

# I am reading this to mean that any mouse ID with duplicated time points is suspect/invalid and all data associated with
# mouse ID must be removed from the file

# Looking at data for mouse with duplicated timepoints reinforces this interpretation because column values vary by
# timepoint, indicating that Mouse ID was entered incorrectly (duplicated another mouse), or there was something wrong with
# the way that the data was collected/entered for this ID with duplicated timepoints

In [56]:
# Getting the duplicate mice by ID number that shows up for Mouse ID and Timepoint. 
# Create a group by on ID and Timepoint - any Mouse ID/timepoint with counts greater than 1 indicates duplicates
mouse_group = mouse_data_merged.groupby(["Mouse ID", "Timepoint"])
df = mouse_group.count() 
df = df.loc[df["Drug Regimen"]>1, :]

# Create a list of mouse IDs with duplicate timepoints
mouse_group2 = df.groupby(["Mouse ID"])
mice_with_duplicated_timepoints = mouse_group2.count()
mice_with_duplicated_timepoints = mice_with_duplicated_timepoints.reset_index()
mice_with_duplicated_timepoints

,Mouse ID,Drug Regimen,Sex,Age_months,Weight (g),Tumor Volume (mm3),Metastatic Sites
0,g989,5,5,5,5,5,5


In [85]:
# Optional: Get all the data for the duplicate mouse ID. 

bad_mice_data = mice_with_duplicated_timepoints.merge(mouse_data_merged, left_on="Mouse ID", right_on="Mouse ID", 
                                                      how="left")
rows_to_drop = len(bad_mice_data)

bad_mice_data

,Mouse ID,Drug Regimen_x,Sex_x,Age_months_x,Weight (g)_x,Tumor Volume (mm3)_x,Metastatic Sites_x,Drug Regimen_y,Sex_y,Age_months_y,Weight (g)_y,Timepoint,Tumor Volume (mm3)_y,Metastatic Sites_y
0,g989,5,5,5,5,5,5,Propriva,Female,21,26,0,45.000000,0
1,g989,5,5,5,5,5,5,Propriva,Female,21,26,0,45.000000,0
2,g989,5,5,5,5,5,5,Propriva,Female,21,26,5,48.786801,0
3,g989,5,5,5,5,5,5,Propriva,Female,21,26,5,47.570392,0
4,g989,5,5,5,5,5,5,Propriva,Female,21,26,10,51.745156,0
5,g989,5,5,5,5,5,5,Propriva,Female,21,26,10,49.880528,0
6,g989,5,5,5,5,5,5,Propriva,Female,21,26,15,51.325852,1
7,g989,5,5,5,5,5,5,Propriva,Female,21,26,15,53.442020,0
8,g989,5,5,5,5,5,5,Propriva,Female,21,26,20,55.326122,1
9,g989,5,5,5,5,5,5,Propriva,Female,21,26,20,54.657650,1


In [87]:
# Create a clean DataFrame by dropping the duplicate mouse by its ID.
# Outer join mice with duplicates to the orginal merged dataframe
# records with no match on mice with duplicates will have NaN values in the resulting fields from mice with duplicates
clean_mice_data = mice_with_duplicated_timepoints.merge(mouse_data_merged, left_on="Mouse ID", right_on="Mouse ID", 
                                                        how="outer", suffixes=("_bad",""))

# count rows before dropping mice with duplicates
rows_before_drop = len(clean_mice_data)

# identify rows to keep: rows that had no match in the outer join to mice_with_duplicated_timepoints
# drop rows that have non-zero values after replacing NaN with zero
#
# OBS!  This works in this instance, but not a good practice to continue.  At least not without additional
# data cleaning to ensure that all the values in Sex are valid (i.e. there were no zeroes in this column before
# replacing all the NaN values with zero)
#
clean_mice_data = clean_mice_data.fillna(0)
clean_mice_data = clean_mice_data.drop(clean_mice_data[clean_mice_data.Sex_bad >0].index)

# remove duplicated columns with non-values
clean_mice_data = clean_mice_data.drop([col for col in clean_mice_data if col.endswith('_bad')], 1)

# count rows in cleaned dataframe
rows_after_drop = len(clean_mice_data)

clean_mice_data.head()

,Mouse ID,Drug Regimen,Sex,Age_months,Weight (g),Timepoint,Tumor Volume (mm3),Metastatic Sites
13,k403,Ramicane,Male,21,16,0,45.000000,0
14,k403,Ramicane,Male,21,16,5,38.825898,0
15,k403,Ramicane,Male,21,16,10,35.014271,1
16,k403,Ramicane,Male,21,16,15,34.223992,1
17,k403,Ramicane,Male,21,16,20,32.997729,1


In [86]:
# Checking the number of mice in the clean DataFrame.

# print row counts for validation
print(f'Row count before drop: {rows_before_drop}')
print(f'Count of rows to drop: {rows_to_drop}')
print(f'Row count after drop (clean dataframe): {rows_after_drop}')


Row count before drop: 1893
Count of rows to drop: 13
Row count after drop (clean dataframe): 1880


## Summary Statistics

In [7]:
# Generate a summary statistics table of mean, median, variance, standard deviation, and SEM of the tumor volume for each regimen

# Use groupby and summary statistical methods to calculate the following properties of each drug regimen: 
# mean, median, variance, standard deviation, and SEM of the tumor volume. 
# Assemble the resulting series into a single summary dataframe.



In [8]:
# Generate a summary statistics table of mean, median, variance, standard deviation, and SEM of the tumor volume for each regimen

# Using the aggregation method, produce the same summary statistics in a single line


## Bar and Pie Charts

In [9]:
# Generate a bar plot showing the total number of unique mice tested on each drug regimen using pandas.



In [10]:
# Generate a bar plot showing the total number of unique mice tested on each drug regimen using pyplot.



In [11]:
# Generate a pie plot showing the distribution of female versus male mice using pandas



In [12]:
# Generate a pie plot showing the distribution of female versus male mice using pyplot



## Quartiles, Outliers and Boxplots

In [13]:
# Calculate the final tumor volume of each mouse across four of the treatment regimens:  
# Capomulin, Ramicane, Infubinol, and Ceftamin

# Start by getting the last (greatest) timepoint for each mouse


# Merge this group df with the original dataframe to get the tumor volume at the last timepoint


In [14]:
# Put treatments into a list for for loop (and later for plot labels)


# Create empty list to fill with tumor vol data (for plotting)


# Calculate the IQR and quantitatively determine if there are any potential outliers. 

    
    # Locate the rows which contain mice on each drug and get the tumor volumes
    
    
    # add subset 
    
    
    # Determine outliers using upper and lower bounds
    

In [15]:
# Generate a box plot of the final tumor volume of each mouse across four regimens of interest


## Line and Scatter Plots

In [16]:
# Generate a line plot of tumor volume vs. time point for a mouse treated with Capomulin


In [17]:
# Generate a scatter plot of average tumor volume vs. mouse weight for the Capomulin regimen


## Correlation and Regression

In [18]:
# Calculate the correlation coefficient and linear regression model 
# for mouse weight and average tumor volume for the Capomulin regimen
